## Reducing Network Latency & Anomaly Detecting using an LSTM Encoder-Decoder Model

The goal of this project is twofold:
- Reduce the network traffic in the cloud from the gateway layer.
- Detect anomalous data, indicating a faulty sensor or a potential attack.

We use a subset of data collected from Intel Labs between March and April, 2004 (http://db.csail.mit.edu/labdata/labdata.htmlO) as a proof of concept for applying Deep Learning at the IoT Gateway Layer.

In the best case scenario, we can send predicted batches of time series data that are representative of the actual readings in $n/k$ transmissions, where $n$ is the size of our time series set, and $k$ is the batching size.

In the worst case scenario, incorrectly predicted batch values will update what is currently in the cloud at the time of sensor reading. This scenario will perform as well as trivially passing data from the gateway to the cloud unhindered in $n$ transmissions.

### Data Loading

In [1]:
import gzip
import pandas as pd

In [2]:
with gzip.open('data.txt.gz', 'rb') as data_bytes:
    data = pd.read_csv(data_bytes, header=None, sep=' ', parse_dates=[[0, 1]], squeeze=True)
data.columns = ['DATETIME','EPOCH','SENSOR_ID','TEMPERATURE','HUMIDITY','LIGHT','VOLTAGE']
data = data.set_index('DATETIME')

In [3]:
data.shape

(2313682, 6)

### Data Pre-processing

We will consider sensor data between March 1st and March 10th for this experiment, as it contains the majority of the complete data.

In [4]:
data_samp = data.loc['2004-03-01':'2004-03-10'].copy()
data_samp.shape

(892574, 6)

For the purposes of a proof of concept, we will make this a univariate problem (not including DateTime), focusing on Temperature readings.

In [5]:
data_samp.drop(['HUMIDITY','LIGHT','VOLTAGE','EPOCH'], axis=1, inplace=True)

Dropping any Sensor ID's where the value is NaN.

In [6]:
data_samp.dropna(subset=['SENSOR_ID'], inplace=True)

For the sake of out experiment, let us only consider sensors 1-10.

In [7]:
data_samp = data_samp[(data_samp.SENSOR_ID >= 1) & (data_samp.SENSOR_ID <= 10)]

Reshaping the Sensor ID field to an integer value.

In [8]:
data_samp.SENSOR_ID.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [9]:
data_samp.SENSOR_ID = data_samp.SENSOR_ID.astype(int)
data_samp.dtypes

SENSOR_ID        int64
TEMPERATURE    float64
dtype: object

In [10]:
data_samp.head()

,SENSOR_ID,TEMPERATURE
DATETIME,,
2004-03-01 00:01:57.130850,1,18.4498
2004-03-01 00:02:50.458234,1,18.4400
2004-03-01 00:04:26.606602,1,18.4400
2004-03-01 00:05:28.379208,1,18.4498
2004-03-01 00:05:50.456126,1,18.4302


We want to measure the temperature at each sensor for a given timestamp, so we will pivot the table, making the column values sensor temperature readings at a given timestamp.

In [11]:
data_samp = data_samp.pivot(columns='SENSOR_ID', values='TEMPERATURE')

In [12]:
data_samp.head()

SENSOR_ID,1,2,3,4,5,6,7,8,9,10
DATETIME,,,,,,,,,,
2004-03-01 00:00:21.445722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.489,NaN
2004-03-01 00:00:22.429139,NaN,18.8712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-01 00:00:25.633782,NaN,NaN,NaN,NaN,NaN,NaN,18.7144,NaN,NaN,NaN
2004-03-01 00:00:52.381230,NaN,18.8614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-01 00:00:53.317719,NaN,NaN,NaN,NaN,NaN,NaN,18.7046,NaN,NaN,NaN


There appears to be a lot of missing values for temperature readings in our table, due to micro-second DateTime ID's in our time series set. We will resample the data every 2 minutes, taking the mean of the values collected.

In [13]:
data_samp = data_samp.resample('2min').mean()

In [14]:
print('New resampled set has: {} data points.'.format(len(data_samp)))
data_samp.isna().sum()

New resampled set has: 6754 data points.


SENSOR_ID
1      143
2      629
3      114
4      349
5     6754
6      582
7       17
8      753
9       65
10     163
dtype: int64

Clearly sensor 5 is not reading values between our time frame, so we will drop it. Stack brings the prescribed column (SENSOR_ID) into our index, making it easily dropped. We unstack to bring Sensor ID out of the index.

In [15]:
temp_df = data_samp.stack().drop(5, level='SENSOR_ID')
data_samp = temp_df.unstack()

In [16]:
data_samp

SENSOR_ID,1,2,3,4,6,7,8,9,10
DATETIME,,,,,,,,,
2004-03-01 00:00:00,18.449800,18.864667,18.753600,19.11130,18.6752,18.70705,18.386100,18.484100,18.430200
2004-03-01 00:02:00,18.440000,18.848333,18.756867,19.10640,18.6654,18.69235,18.378750,18.469400,NaN
2004-03-01 00:04:00,18.440000,18.832000,18.734000,19.10640,18.6654,18.68500,18.376300,18.475933,18.400800
2004-03-01 00:06:00,NaN,18.851600,18.753600,19.10640,18.6654,18.67765,18.377933,18.482467,18.410600
2004-03-01 00:08:00,18.435100,18.861400,18.773200,19.10150,18.6556,18.68500,18.371400,18.479200,18.433467
...,...,...,...,...,...,...,...,...,...
2004-03-10 08:58:00,22.835300,23.178300,23.776100,23.93045,24.0946,23.99170,25.296733,26.044800,24.473533
2004-03-10 09:00:00,22.879400,23.134200,23.717300,23.92800,24.1044,23.92800,25.395550,26.113400,24.589500
2004-03-10 09:02:00,22.869600,23.121950,23.676467,23.90840,24.1485,23.95740,25.496000,26.280000,24.692400


There are still some missing values, which we can simply deal with by applying linear interpolation to estimate values making our set continuous. Interpolation uses previous values, so for values appearing at the front of our frame (ie. sensor 1) we must make the process bidirectional.

In [17]:
data_samp = data_samp.interpolate(method='linear', limit_direction='both', axis=0)

In [18]:
data_samp.isna().sum()

SENSOR_ID
1     0
2     0
3     0
4     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

In [19]:
data_samp.describe()

SENSOR_ID,1,2,3,4,6,7,8,9,10
count,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000
mean,22.192462,22.126009,22.240772,22.249970,21.786615,21.844309,21.621812,21.801295,21.549061
std,2.395218,1.944178,2.198261,2.049267,1.874288,1.955498,2.174163,2.258517,1.976967
min,17.195400,17.642933,17.577600,18.038200,17.616800,17.789933,10.487300,17.499200,17.548200
25%,20.581300,20.881425,20.767500,20.988000,20.547612,20.574563,20.090075,20.143362,20.174600
50%,22.041500,22.259142,22.213000,22.046400,21.821000,21.742600,21.715650,21.811200,21.613567
75%,23.869200,23.349800,23.709133,23.437387,23.166867,23.178300,22.908800,23.152983,22.813250
max,28.654867,27.416800,28.243267,27.652000,26.534800,26.420467,26.453950,27.162000,25.819400


Now our data set smoothly tracks Temperature over a 2 minute interval without undefined data points. Let's plot our findings for each sensor in our dataframe.

In [20]:
import matplotlib.pyplot as plt

data_samp.plot(subplots=True, legend=True, figsize=(10,20))

array([<matplotlib.axes._subplots.AxesSubplot object at 0x11a7519d0>,
      dtype=object)

### LSTM Encoder-Decoder

Given an array of 9 sensor values with readings at every 2 minute interval, we would like to generate a compressed representation of these values.

Consider an input vector to out encoder of size 9, that takes the following form:

$\vec{S}=\lbrace\langle s_1, \cdots, s_{n} \rangle | s_k\in S, 1\leq k\leq n\rbrace$. Where $S$ is the set of sensors in our network that can transmit data to a gateway.

Our autoencoder will attempt to encode a representation of $\vec{S}$, shown as $h$ below, and use it to detect anomalies.

![TEST](./img/autoencoder.png)

We are spatially clustering these sensors based on physical proximity to one another, but the overall size of $\vec{S}$ is arbitrary.

In [21]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, TimeDistributed
from keras.utils import plot_model

Using TensorFlow backend.


First, we will split our train/test sets. We will choose a 90/10 split.

In [23]:
train, test = train_test_split(data_samp, test_size=0.1)
print('Training shape: {}, Testing shape: {}'.format(train.shape, test.shape))

Training shape: (6078, 9), Testing shape: (676, 9)


In [49]:
# Arbitrarily chosen look_back value
look_back = 5
n_features = train.shape[1]
n_samples_train = train.shape[0] - lookback
n_samples_test = test.shape[0] - lookback

In [48]:
train_reshape = np.zeros((n_samples_train, look_back, n_features))
test_reshape = np.zeros((n_samples_test, look_back, n_features))

for i in range(n_samples_train):
    train_reshape[i] = train[i:i+look_back]

for j in range(n_samples_test):
    test_reshape[j] = test[j:j+look_back]

print('Reshaped Train: {}, reshaped Test: {}'.format(train_reshape.shape, test_reshape.shape))

Reshaped Train: (6073, 5, 9), reshaped Test: (671, 5, 9)


In [57]:
def build_model(reshaped_data):
    model = Sequential()
    model.add(LSTM(units=128, activation='relu', input_shape=(reshaped_data.shape[1], reshaped_data.shape[2])))
    model.add(RepeatVector(reshaped_data.shape[1]))
    model.add(LSTM(units=128, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(units=reshaped_data.shape[2])))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [61]:
model = build_model(train_reshape)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 128)               70656     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 9)              1161      
Total params: 203,401
Trainable params: 203,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Split the test data subset into an even smaller validation split for scoring.

In [64]:
history = model.fit(
    train_reshape, train_reshape,
    epochs=10,
    batch_size=32,
    validation_split=0.1
)

Train on 5465 samples, validate on 608 samples
Epoch 1/10
5465/5465 [==============================] - 6s 1ms/step - loss: 29.1600 - val_loss: 2.6725
Epoch 2/10
5465/5465 [==============================] - 5s 889us/step - loss: 2.0432 - val_loss: 1.3379
Epoch 3/10
5465/5465 [==============================] - 5s 896us/step - loss: 0.9571 - val_loss: 0.4820
Epoch 4/10
5465/5465 [==============================] - 5s 909us/step - loss: 0.3871 - val_loss: 0.3485
Epoch 5/10
5465/5465 [==============================] - 5s 906us/step - loss: 0.3093 - val_loss: 0.2807
Epoch 6/10
5465/5465 [==============================] - 5s 922us/step - loss: 0.2623 - val_loss: 0.2388
Epoch 7/10
5465/5465 [==============================] - 6s 1ms/step - loss: 0.2545 - val_loss: 0.2151
Epoch 8/10
5465/5465 [==============================] - 5s 971us/step - loss: 0.2404 - val_loss: 0.2380
Epoch 9/10
5465/5465 [==============================] - 5s 905us/step - loss: 0.2220 - val_loss: 0.2001
Epoch 10/10
5465/546